In [1]:
import os
import shutil

# Temporary Directory
tmp = '/kaggle/tmp/'
# os.mkdir(tmp)

repo_name = 'AudioLDM-training-finetuning'

tmp_sub_dir = tmp + repo_name
os.makedirs(tmp_sub_dir)

In [2]:
os.listdir(tmp)

['AudioLDM-training-finetuning']

### Formatting 

Cloning github to temporary directory. Specifically to /kaggle/tmp/AudioLDM-training-finetuning

In [3]:
# !git clone https://github.com/haoheliu/AudioLDM-training-finetuning.git /kaggle/tmp/AudioLDM-training-finetuning

In [4]:
# cloned = '/kaggle/tmp/AudioLDM-training-finetuning'

# os.listdir(cloned)

In [5]:
dataset_source = '/kaggle/input/project03'
dataset_destination = tmp_sub_dir

# Copy the dataset directory recursively
shutil.copytree(dataset_source, dataset_destination, dirs_exist_ok=True)

# os.rename(dataset_destination + '/' + 'project03', dataset_destination + '/' + repo_name)

# dataset_source = '/kaggle/input/dcase2024-task-7/dcase_2024_task7'
# dataset_destination = '/kaggle/tmp/AudioLDM-training-finetuning/data/dataset'

# # Copy the dataset directory recursively
# shutil.copytree(dataset_source, dataset_destination, dirs_exist_ok=True)

'/kaggle/tmp/AudioLDM-training-finetuning'

In [6]:
# checkpoints_source = '/kaggle/input/checkpoint/checkpoints'
# checkpoints_destination = '/kaggle/tmp/AudioLDM-training-finetuning/data/checkpoints'

# # Copy the checkpoints directory recursively
# shutil.copytree(checkpoints_source, checkpoints_destination, dirs_exist_ok=True)

In [7]:
# List contents of the dataset directory
!ls /kaggle/tmp/AudioLDM-training-finetuning

LICENSE    audioldm_train  bash_train.sh  log	       pyproject.toml  tests
README.md  bash_eval.sh    data		  poetry.lock  taming


In [8]:
# List contents of the checkpoints directory
!ls /kaggle/tmp/AudioLDM-training-finetuning/data/checkpoints

audiomae_16k_128bins.ckpt			  hifigan_16k_64bins.json
clap_htsat_tiny.pt				  hifigan_48k_256bins.ckpt
clap_music_speech_audioset_epoch_15_esc_89.98.pt  hifigan_48k_256bins.json
hifigan_16k_64bins.ckpt				  vae_mel_16k_64bins.ckpt


In [9]:
# Navigate to the root of the cloned repository
%cd /kaggle/tmp/AudioLDM-training-finetuning

/kaggle/tmp/AudioLDM-training-finetuning


In [10]:
# Run the validation script
!python3 tests/validate_dataset_checkpoint.py

All files and directories are present
Checking the validity of the audio datasets
100%|█████████████████████████████████████| 964/964 [00:00<00:00, 133091.15it/s]
All audio files are present. You are good to go!


### Setting up the Python Environment

In [11]:
%cd /kaggle/tmp/AudioLDM-training-finetuning

/kaggle/tmp/AudioLDM-training-finetuning


instaling the packages in editable model looks for setup.py or pyproject.toml file in the current directory and installs the packages.

In [12]:
!pip install -e .

Obtaining file:///kaggle/tmp/AudioLDM-training-finetuning
  Installing build dependencies ... - \ | done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/haoheliu/audioldm_eval.git to /tmp/pip-install-uiuntigl/audioldm-eval_2e2a7d540542466386753cbbc9b51776
  Running command git clone --filter=blob:none --quiet https://github.com/haoheliu/audioldm_eval.git /tmp/pip-install-uiuntigl/audioldm-eval_2e2a7d540542466386753cbbc9b51776
  Resolved https://github.com/haoheliu/audioldm_eval.git to commit 8dc07ee7c42f9dc6e295460a1034175a0d49b436
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/haoheliu/passt_hear21.git to /tmp/pip-install-uiuntigl/hear21passt_55003c084cc9473fa5f0eaf241193fdd
  Running command git clone --filter=blob:none --quiet https://github.com/haoheliu/passt_hear21.git /tmp/pip-install-uiuntigl/hear21passt_

In [13]:
# Try importing the module to check if it's installed correctly
try:
    import audioldm_train
    print("Module 'audioldm_train' imported successfully.")
except ImportError as e:
    print(f"ImportError: {e}")

Module 'audioldm_train' imported successfully.


# **RUN THIS B4 TRRAINING**

In [14]:
!pip install librosa==0.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Successfully uninstalled librosa-0.10.2.post1


### Running Training Script

In [15]:
# Ensure you're in the root directory
%cd /kaggle/tmp/AudioLDM-training-finetuning

/kaggle/tmp/AudioLDM-training-finetuning


In [16]:
str_to_write = '''
metadata_root: "./data/dataset/metadata/dataset_root.json"
log_directory: "./log/latent_diffusion"
project: "audioldm"
precision: "high"

variables:
  sampling_rate: &sampling_rate 16000 
  mel_bins: &mel_bins 64
  latent_embed_dim: &latent_embed_dim 8
  latent_t_size: &latent_t_size 256 # TODO might need to change
  latent_f_size: &latent_f_size 16
  in_channels: &unet_in_channels 8
  optimize_ddpm_parameter: &optimize_ddpm_parameter true
  optimize_gpt: &optimize_gpt true
  warmup_steps: &warmup_steps 2000

data: 
  train: ["audiocaps"]
  val: "audiocaps"
  test: "audiocaps"
  class_label_indices: "audioset_eval_subset"
  dataloader_add_ons: [] 

step:
  validation_every_n_epochs: 10
  save_checkpoint_every_n_steps: 45000
  # limit_val_batches: 2
  max_steps: 45000
  save_top_k: 1

preprocessing:
  audio:
    sampling_rate: *sampling_rate
    max_wav_value: 32768.0
    duration: 10.24
  stft:
    filter_length: 1024
    hop_length: 160
    win_length: 1024
  mel:
    n_mel_channels: *mel_bins
    mel_fmin: 0
    mel_fmax: 8000 

augmentation:
  mixup: 0.0

model:
  target: audioldm_train.modules.latent_diffusion.ddpm.LatentDiffusion
  params: 
    # Autoencoder
    first_stage_config:
      base_learning_rate: 8.0e-06
      target: audioldm_train.modules.latent_encoder.autoencoder.AutoencoderKL
      params: 
        reload_from_ckpt: "data/checkpoints/vae_mel_16k_64bins.ckpt"
        sampling_rate: *sampling_rate
        batchsize: 4
        monitor: val/rec_loss
        image_key: fbank
        subband: 1
        embed_dim: *latent_embed_dim
        time_shuffle: 1
        lossconfig:
          target: audioldm_train.losses.LPIPSWithDiscriminator
          params:
            disc_start: 50001
            kl_weight: 1000.0
            disc_weight: 0.5
            disc_in_channels: 1
        ddconfig: 
          double_z: true
          mel_bins: *mel_bins # The frequency bins of mel spectrogram
          z_channels: 8
          resolution: 256
          downsample_time: false
          in_channels: 1
          out_ch: 1
          ch: 128 
          ch_mult:
          - 1
          - 2
          - 4
          num_res_blocks: 2
          attn_resolutions: []
          dropout: 0.0
    
    # Other parameters
    base_learning_rate: 1.0e-4
    warmup_steps: *warmup_steps
    optimize_ddpm_parameter: *optimize_ddpm_parameter
    sampling_rate: *sampling_rate
    batchsize: 2
    linear_start: 0.0015
    linear_end: 0.0195
    num_timesteps_cond: 1
    log_every_t: 200
    timesteps: 1000
    unconditional_prob_cfg: 0.1
    parameterization: eps # [eps, x0, v]
    first_stage_key: fbank
    latent_t_size: *latent_t_size # TODO might need to change
    latent_f_size: *latent_f_size
    channels: *latent_embed_dim # TODO might need to change
    monitor: val/loss_simple_ema
    scale_by_std: true
    unet_config:
      target: audioldm_train.modules.diffusionmodules.openaimodel.UNetModel
      params:
        image_size: 64 
        extra_film_condition_dim: 512 # If you use film as extra condition, set this parameter. For example if you have two conditioning vectors each have dimension 512, then this number would be 1024
        # context_dim: 
        # - 768
        in_channels: *unet_in_channels # The input channel of the UNet model
        out_channels: *latent_embed_dim # TODO might need to change
        model_channels: 128 # TODO might need to change
        attention_resolutions:
        - 8
        - 4
        - 2
        num_res_blocks: 2
        channel_mult: 
        - 1
        - 2
        - 3
        - 5
        num_head_channels: 32
        use_spatial_transformer: true
        transformer_depth: 1
        extra_sa_layer: false
    
    cond_stage_config:
      film_clap_cond1:
        cond_stage_key: text
        conditioning_key: film
        target: audioldm_train.conditional_models.CLAPAudioEmbeddingClassifierFreev2
        params:
          pretrained_path: data/checkpoints/clap_htsat_tiny.pt
          sampling_rate: 16000
          embed_mode: text # or text
          amodel: HTSAT-tiny

    evaluation_params:
      unconditional_guidance_scale: 3.5
      ddim_sampling_steps: 200
      n_candidates_per_samples: 3
'''

yaml_path = '/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml'

# os.listdir('/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/config/2023_08_23_reproduce_audioldm')
with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(str_to_write)

In [17]:
!cat /kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml


metadata_root: "./data/dataset/metadata/dataset_root.json"
log_directory: "./log/latent_diffusion"
project: "audioldm"
precision: "high"

variables:
  sampling_rate: &sampling_rate 16000 
  mel_bins: &mel_bins 64
  latent_embed_dim: &latent_embed_dim 8
  latent_t_size: &latent_t_size 256 # TODO might need to change
  latent_f_size: &latent_f_size 16
  in_channels: &unet_in_channels 8
  optimize_ddpm_parameter: &optimize_ddpm_parameter true
  optimize_gpt: &optimize_gpt true
  warmup_steps: &warmup_steps 2000

data: 
  train: ["audiocaps"]
  val: "audiocaps"
  test: "audiocaps"
  class_label_indices: "audioset_eval_subset"
  dataloader_add_ons: [] 

step:
  validation_every_n_epochs: 10
  save_checkpoint_every_n_steps: 45000
  # limit_val_batches: 2
  max_steps: 45000
  save_top_k: 1

preprocessing:
  audio:
    sampling_rate: *sampling_rate
    max_wav_value: 32768.0
    duration: 10.24
  stft:
    filter_length: 1024
    hop_length: 160
    win_length: 1024
  mel:
    n_mel_channels:

In [18]:
import os

# Disable wandb
os.environ['WANDB_MODE'] = 'disabled'

import gzip

original_file = '/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/modules/clap/open_clip/bpe_simple_vocab_16e6.txt'
target_file = '/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/modules/clap/open_clip/bpe_simple_vocab_16e6.txt.gz'

with open(original_file, 'rb') as f_in, gzip.open(target_file, 'wb') as f_out:
    f_out.writelines(f_in)

# Run the training script
!python audioldm_train/train/latent_diffusion.py -c audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml
# !python3 audioldm_train/train/autoencoder.py -c audioldm_train/config/2023_11_13_vae_autoencoder/16k_64.yaml

SEED EVERYTHING TO 0
Add-ons: []
Build dataset split train from ['audiocaps']
Data size: 49502
/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/utilities/audio/stft.py:42: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/utilities/audio/stft.py:145: FutureWarning: Pass sr=16000, n_fft=1024, n_mels=64, fmin=0, fmax=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(
Dataset initialize finished
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get Data

In [19]:
# !python3 audioldm_train/infer.py --config_yaml audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml --list_inference tests/captionlist/inference_test_with_filename.lst --reload_from_ckpt log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoints/checkpoint-fad-133.00-global_step=49.ckpt

In [20]:
print(tmp_sub_dir)
os.listdir(tmp_sub_dir + '/log/latent_diffusion/2023_08_23_reproduce_audioldm')

/kaggle/tmp/AudioLDM-training-finetuning


['audioldm_original', 'audioldm_crossattn_flant5']

In [21]:
!ls -la /kaggle/tmp/AudioLDM-training-finetuning/log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original

total 24
drwxr-xr-x 4 root root 4096 Nov 22 21:02 .
drwxr-xr-x 4 root root 4096 Nov 22 21:02 ..
-rw-r--r-- 1 root root 4211 Nov 22 21:02 audioldm_original.yaml
drwxr-xr-x 2 root root 4096 Nov 23 08:02 checkpoints
drwxr-xr-x 2 root root 4096 Nov 22 21:10 infer_11-22-21:02_cfg_scale_3.5_ddim_200_n_cand_3


In [22]:
ckpt_name = os.listdir('/kaggle/tmp/AudioLDM-training-finetuning/log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoints')[0]
ckpt_path = '/kaggle/tmp/AudioLDM-training-finetuning/log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/' + ckpt_name
print(ckpt_path)
os.system(f'python3 audioldm_train/infer.py --config_yaml audioldm_train/config/2023_08_23_reproduce_audioldm/audioldm_original.yaml --list_inference tests/captionlist/inference_test_with_filename.lst --reload_from_ckpt {ckpt_path}')

/kaggle/tmp/AudioLDM-training-finetuning/log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoint-fad-133.00-global_step=44999.ckpt


/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/utilities/audio/stft.py:42: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/kaggle/tmp/AudioLDM-training-finetuning/audioldm_train/utilities/audio/stft.py:145: FutureWarning: Pass sr=16000, n_fft=1024, n_mels=64, fmin=0, fmax=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. 

SEED EVERYTHING TO 0
Add-ons: []
Dataset initialize finished
Load checkpoint from path: ./log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoints
Resume from checkpoint ./log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/checkpoints/checkpoint-fad-133.00-global_step=44999.ckpt
LatentDiffusion: Running in eps-prediction mode
+ Use extra condition on UNet channel using Film. Extra condition dimension is 512. 
DiffusionWrapper has 185.04 M params.
Keeping EMAs of 692.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
Removing weight norm...
Initial learning rate 1e-05
--> Reload weight of autoencoder from data/checkpoints/vae_mel_16k_64bins.ckpt
Waveform inference save path:  ./log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original/infer_11-23-08:04_cfg_scale_3.5_ddim_200_n_cand_3
Plotting: Switch

DDIM Sampler: 100%|██████████| 200/200 [01:10<00:00,  2.86it/s]


The shape of UNet input is torch.Size([3, 8, 256, 16])
INFO: clap model calculate the audio embedding as condition
Similarity between generated audio and text tensor([ 0.1145,  0.0122, -0.0720], device='cuda:0')
Choose the following indexes: [0]
Non-fatal Warning [dataset.py]: The wav path " audiofile02.wav " is not find in the metadata. Use empty waveform instead. This is normal in the inference process.
Use ddim sampler
Data shape for DDIM sampling is (3, 8, 256, 16), eta 1.0
Running DDIM Sampling with 200 timesteps


...

DDIM Sampler: 100%|██████████| 200/200 [01:14<00:00,  2.69it/s]


INFO: clap model calculate the audio embedding as condition
Similarity between generated audio and text tensor([-0.0103,  0.1107, -0.1062], device='cuda:0')
Choose the following indexes: [1]
Non-fatal Warning [dataset.py]: The wav path " audiofile06.wav " is not find in the metadata. Use empty waveform instead. This is normal in the inference process.
Use ddim sampler
Data shape for DDIM sampling is (3, 8, 256, 16), eta 1.0
Running DDIM Sampling with 200 timesteps


DDIM Sampler: 100%|██████████| 200/200 [01:13<00:00,  2.71it/s]


INFO: clap model calculate the audio embedding as condition
Similarity between generated audio and text tensor([0.1712, 0.1284, 0.1977], device='cuda:0')
Choose the following indexes: [2]
Plotting: Restored training weights


0

In [23]:
shutil.copytree('/kaggle/tmp/AudioLDM-training-finetuning/log/latent_diffusion/2023_08_23_reproduce_audioldm/audioldm_original', '/kaggle/working/', dirs_exist_ok=True)

'/kaggle/working/'